In [1]:
import os
os.chdir('../..')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.linear_model import LinearRegression
from collections import Iterable
import random
from os import listdir
from os.path import isfile, join
import matplotlib.patches as mpatches
import pickle
import glob

from itertools import combinations

import seaborn as sns
sns.set_style("whitegrid")
from CTABGAN.model.eval.evaluation import get_utility_metrics,stat_sim,privacy_metrics

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

D:\Users\glede\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


In [3]:
dataset = 'LPMC'
orig_path = '../data/' + dataset + '/data.csv'

In [4]:
input_folder = '../synth_data/TEST/'

files_ = {}
models = []

for f in listdir(input_folder):
    if isfile(join(input_folder, f)):
        m = f.split('.')[0]
        models.append(m)
        files_[m] = join(input_folder, f)

In [5]:
for s in files_:
    classifiers_list = ["lr","dt","rf","mlp"]
    result_mat = get_utility_metrics(orig_path, [files_[s]], "MinMax", classifiers_list, test_ratio = 0.20, target='travel_mode')
    result_df  = pd.DataFrame(result_mat, columns=["Acc","AUC","F1_Score"])
    result_df.index = classifiers_list
    print(s)
    print(result_df)
    print()

CTABGAN
           Acc       AUC  F1_Score
lr   14.387060  0.037495  0.126386
dt   12.712826  0.065168  0.114746
rf    8.115778  0.043870  0.072341
mlp  13.138479  0.050769  0.115915

CTGAN
           Acc       AUC  F1_Score
lr    3.064699  0.013369  0.028684
dt   16.572077  0.133681  0.172254
rf    3.234960  0.031216  0.030745
mlp   3.916005  0.023781  0.037868

DATGAN
          Acc       AUC  F1_Score
lr   3.972758  0.011659  0.033527
dt   7.945516  0.050679  0.076548
rf   3.859251  0.022573  0.036035
mlp  4.029512  0.019956  0.041387



In [6]:
if dataset is 'Chicago':
    discrete_columns = [
        'choice',
        'travel_dow',
        'trip_purpose',
        'hh_vehicles',
        'hh_size',
        'hh_bikes',
        'hh_descr',
        'hh_income',
        'gender',
        'license',
        'education_level',
        'work_status'
    ]
elif dataset is 'LPMC':
    discrete_columns = [
        'travel_mode',
        'purpose',
        'fueltype',
        'faretype',
        'bus_scale',
        'travel_year',
        'travel_month',
        'travel_date',
        'day_of_week',
        'female',
        'driving_license',
        'car_ownership',
        'pt_n_interchanges',
        'cost_driving_con_charge'
    ]

stat_res_avg = []
for s in files_:
    stat_res = stat_sim(orig_path,files_[s],discrete_columns)
    stat_res_avg.append(stat_res)

stat_columns = ["Average WD (Continuous Columns","Average JSD (Categorical Columns)","Correlation Distance"]
stat_results = pd.DataFrame(np.array(stat_res_avg),columns=stat_columns)
stat_results.index = files_.keys()
print(stat_results)

         Average WD (Continuous Columns  Average JSD (Categorical Columns)  \
CTABGAN                        0.029011                           0.121527   
CTGAN                          0.022925                           0.097058   
DATGAN                         0.007726                           0.019094   

         Correlation Distance  
CTABGAN              2.713892  
CTGAN                1.899819  
DATGAN               1.730790  


In [7]:
priv_res_avg = []
for s in files_:
    priv_res = privacy_metrics(orig_path,files_[s])
    priv_res_avg.append(priv_res)
    
privacy_columns = ["DCR between Real and Fake (5th perc)","DCR within Real(5th perc)","DCR within Fake (5th perc)","NNDR between Real and Fake (5th perc)","NNDR within Real (5th perc)","NNDR within Fake (5th perc)"]
privacy_results = pd.DataFrame(np.array(priv_res_avg).reshape(3,6),columns=privacy_columns)
privacy_results.index = files_.keys()
print(privacy_results)

         DCR between Real and Fake (5th perc)  DCR within Real(5th perc)  \
CTABGAN                              2.903798                   1.625665   
CTGAN                                1.849950                   1.625665   
DATGAN                               1.708961                   1.625665   

         DCR within Fake (5th perc)  NNDR between Real and Fake (5th perc)  \
CTABGAN                    1.685415                               0.876397   
CTGAN                      1.833297                               0.764490   
DATGAN                     1.713483                               0.728039   

         NNDR within Real (5th perc)  NNDR within Fake (5th perc)  
CTABGAN                     0.716443                     0.728822  
CTGAN                       0.716443                     0.749853  
DATGAN                      0.716443                     0.736681  
